In [38]:
from dotenv import load_dotenv

# 토큰 정보 로드
load_dotenv()

True

In [39]:
from typing import Dict, TypedDict

from langchain_core.messages import BaseMessage
import base64


class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        keys: A dictionary where each key is a string.
    """

    keys: Dict[str, any]

In [40]:
import json
import matplotlib.pyplot as plt
import operator
from typing import Annotated, Sequence, TypedDict

from langchain import hub
from langchain.output_parsers.openai_tools import PydanticToolsParser
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores.chroma import Chroma
from langchain_core.messages import BaseMessage, FunctionMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.utils.function_calling import convert_to_openai_tool
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.tools.sql_database.tool import QuerySQLCheckerTool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from IPython.display import Image, display
from langchain_community.chat_models import ChatOllama
import zlib
    
### Nodes ###
def summarize_user_request(state):
    """
    Summarize the user request.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates request key with summarized request
    """


    state_dict = state["keys"]
    request = state_dict["request"]
    language = state_dict["language"]

    prompt = PromptTemplate(
        template="""A step-by-step process is given by the request. You should summarize step by step in chronological order while maintaining important information.

        IMPORTANT: Translate into {language}
        example: 
          request: When the user sends a use request to the client, the client requests the authorization server for an authorization approval code. In this case, parameters included in the request are client_id, redirect_url, and response_type. Simultaneously with the previous process, when the user requests a login to the authorization server, the authorization server transmits the authorization approval code to the client.
          summarized: 1. user sends a use request to the client
                      2. ....

        user request is "{request}"
        IMPORTANT: Just print out summarized result, nothing else.
        """,
        input_variables=["request", "language"]
    )

    # LLM gpt-3.5-turbo
    llm = ChatOpenAI(model_name="gpt-4-0125-preview", temperature=0)
    # llm = ChatOllama(model="mistral:latest")

    # Chain
    chain = prompt | llm | StrOutputParser()

    # Run
    summarized = chain.invoke({"request": request})
    return {
        "keys": {"request": summarized, "language": language}
    }

def convert_to_mermaidjs_code(state):
    """
    Convert summarized user request to mermaidjs code for making a sequence diagram.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates mermaid key with coverted code
    """

    prompt = PromptTemplate(
      template="""The goal is to craft a prompt that enables the creation of a sequence diagram using MermaidJS syntax, as defined by the documentation and examples from the MermaidJS official site. This prompt should guide the generation process to ensure it includes all necessary elements for a comprehensive sequence diagram. To achieve this, the prompt will encapsulate instructions on defining participants, illustrating message exchanges, and utilizing MermaidJS's advanced features such as activations, loops, notes, break, and conditional interactions.

      Instructions:
      - Begin by defining the diagram type to ensure MermaidJS interprets it as a sequence diagram.
      - Clearly list all participants in the interaction, ensuring each is uniquely identified.
      - Detail the sequence of messages exchanged between participants, using arrows to denote direction and message type (solid for direct messages, dotted for responses, etc.).
      - Use 'Notes' for parameters or additional explanations.
      - activation/deactivation (+/-) should be equal to single activate/deactivate instruction.
      - Incorporate advanced diagramming features such as activation bars to indicate when a participant is active, loops for repetitive actions, and conditional blocks to represent decision-making processes.
      - Include comments to explain complex parts of the diagram or to provide additional context.
      - Just print out mermaidjs code, do not user markdown. Your answer starts with 'sequenceDiagram'.

      user request: {request}
      answer: ? """,
      input_variables=["request"]
    )

    state_dict = state["keys"]
    request = state_dict["request"]
    language = state_dict["language"]

    # LLM
    llm = ChatOpenAI(model_name="gpt-4-0125-preview", temperature=0)
    # llm = ChatOllama(model="mistral:latest")

    # Chain
    chain = prompt | llm | StrOutputParser()

    # Run
    mermaidjs = chain.invoke({"request": request})
    mermaidjs = mermaidjs.replace('mermaidjs', '')
    mermaidjs = mermaidjs.replace('```mermaidjs', '')
    mermaidjs = mermaidjs.replace('```', '')
    return {
        "keys": {"request": request, "language": language, "mermaidjs": mermaidjs}
    }

def make_graph(state):
  """
  Make a graph based on mermaidjs

  Args:
      state (dict): The current graph state

  Returns:
      graph Image URL base64 string
  """

  state_dict = state["keys"]
  request = state_dict["request"]
  mermaidjs = state_dict["mermaidjs"]
  language = state_dict["language"]

  graphbytes = mermaidjs.encode('ascii')
  base64_bytes = base64.b64encode(graphbytes)
  base64_string = base64_bytes.decode('ascii')

  return {
        "keys": {
            "request": request, 
            "language": language,
            "mermaidjs": mermaidjs, 
            "image": "https://mermaid.ink/img/" + base64_string
        }
  }

In [41]:
import pprint

from langgraph.graph import END, StateGraph

workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("summarize_user_request", summarize_user_request)  # summarize_user_request
workflow.add_node("convert_to_mermaidjs_code", convert_to_mermaidjs_code)  # convert_to_mermaidjs_code
workflow.add_node("make_graph", make_graph)  # make_graph

# Build graph
workflow.set_entry_point("summarize_user_request")
workflow.add_edge("summarize_user_request", "convert_to_mermaidjs_code")
workflow.add_edge("convert_to_mermaidjs_code", "make_graph")
workflow.add_edge("make_graph", END)

# Compile
app = workflow.compile()

In [42]:
# Run
inputs = {"keys": {
    "request": "Client는 Authorization Server에 Access Token을 요청합니다. 이 경우에 포함된 파라미터는 grant_type입니다. 이에 응답하여, Authorization Server는 Client에 Access Token을 전달합니다. 이 프로세스가 실패하면, 연결이 종료됩니다. 그런 다음 Client는 Resource Server에 보호된 리소스를 획득하도록 요청하고, Resource Server는 요청된 리소스를 Client에 전달합니다.", 
    "language": "Korean"
    }
  }
for output in app.stream(inputs, {"recursion_limit": 25}):
    for key, value in output.items():
        # Node
        pprint.pprint(f"Node '{key}':")
        # Optional: print full state at each node
        pprint.pprint(value["keys"], indent=2, width=80, depth=None)
    pprint.pprint("\n---\n")

# Final generation
pprint.pprint(value['keys']['mermaidjs'])
pprint.pprint(value['keys']['image'])

"Node 'summarize_user_request':"
{ 'request': '1. Client requests an Access Token from the Authorization '
             'Server, including the parameter grant_type.\n'
             '2. In response, the Authorization Server sends an Access Token '
             'to the Client.\n'
             '3. If the process fails, the connection is terminated.\n'
             '4. The Client then requests protected resources from the '
             'Resource Server.\n'
             '5. The Resource Server delivers the requested resources to the '
             'Client.'}
'\n---\n'
"Node 'convert_to_mermaidjs_code':"
{ 'mermaidjs': 'sequenceDiagram\n'
               '    participant C as Client\n'
               '    participant AS as Authorization Server\n'
               '    participant RS as Resource Server\n'
               '\n'
               '    Note over C, AS: Request for Access Token\n'
               '    C->>+AS: Request Access Token with grant_type\n'
               '    alt successful req